In [2]:
import cvxpy as cp
import numpy as np

Solve
$$
\begin{align*}
\min_{x}& &f(x) \\
\text{subject to}& &c(x) = 0 \\
& &M(x) \succeq 0 
\end{align*}
$$

In [ ]:
class ByrdOmojokunTRSDPSolver:
    """
    This class solves the optimization problem:

    min_x           f(x)        (where f(x) is a linear function)
    subject to      c(x) = 0    (where c(x) is some non-linear function)
                    M(x) >> 0   (where >> means positive semidefinite)
    """
    def __init__(self, eq_cons, num_variables):
        self._tr_radius = 1
        self._alpha = 2
        self._beta = 2
        self._gamma = 2
        self._delta = 2
        self._eta = 2
        self._A = lambda x: np.vstack([c.grad(x).T for c in eq_cons])
        self._c = lambda x: np.vstack([c(x) for c in eq_cons])
        self._num_variables = num_variables
    
    def _find_v(self, x):
        v = cp.Variable(self._num_variables)
        obj = cp.Minimize(cp.sum_squares(self._A(x) @ v + self._c(x)))
        cons = [cp.norm2(v) <= self._beta * self._tr_radius, self._M(v) >> 0]
        prob = cp.Problem(obj, cons)
        prob.solve()
        return prob.value

    def _find_step(self, v, x):
        p = cp.Variable(self._num_variables)
        obj = cp.Minimize(self._f(x + p))
        cons = [self._A(x) @ (p - v) == 0, self._M_no_const(p - v) >> 0, cp.norm2(p) <= self._tr_radius]
        prob = cp.Problem(obj, cons)
        prob.solve()
        return prob.value

    def _merit_function(self, x, mu):
        return self._f(x) + mu * np.linalg.norm(self._c(x), 2)

    def _merit_model(self, x, p, mu):
        return self._f(x) + self._f.grad(x) @ p + mu * np.linalg.norm(self._A(x) @ p + self._c(x), 2)

    def _accept_step(self, x, p, mu):
        ared = self._merit_function(x, mu) - self._merit_function(x + p, mu)
        pred = self._merit_model(np.zeros(self._num_variables), mu) - self._merit_model(p, mu)
        rho = ared / pred
        if rho > self._eta and min_eig(self._M(x + p)) >= -self._delta:
            x += p
            # Check this, increase in trust region radius
            self._tr_radius *= self._alpha
        else:
            self._tr_radius *= self._gamma * np.linalg.norm(p, 2)

    def solve(self, init):
        """
        Solve the problem by taking steps.
        """
        x = init
        for _ in range(self._maxiters):
            v = self._find_v(x)
            p = self._find_step(v, x)
            self._accept_step(x, p, mu)
        return self._x

In [ ]:
# Notation: small font is a (row) vector, capital font is a (symmetric) matrix

class LinConstraint:
    # a @ x + b = 0
    def __init__(self, a, b):
        self._a = a
        self._b = b
    def __call__(self, x):
        return self._a.T @ x + self._b
    def grad(self, x):
        return self._a

class QuadConstraint:
    # x.T @ A @ x + b @ x = 0
    def __init__(self, A, b):
        self._A = A
        self._b = b
    def __call__(self, x):
        return x.T @ self._A @ x + self._b.T @ x
    def grad(self, x):
        return self._b + 2 * self._A @ x

# And what about the M(x) >> 0 constraint?

In [ ]:
class Basis:
    def __init__(self, ops: list[str]):
        self._ops = ops
        self._mapping = {op: i for i, op in enumerate(ops)}
        self._sz = len(ops)
    
    def rank(self, word: str):
        v = self._sz ** (len(word)) - 1
        for i, op in enumerate(reversed(word)):
            v += self._mapping[op] * (self._sz ** i)
        return v

    def unrank(self, pos: int):
        len = 0
        while self._sz ** len - 1 <= pos:
            len += 1
        len -= 1
        pos -= self._sz ** len - 1
        word = [None] * len
        for i in range(len):
            word[i] = self._ops[pos % self._sz]
            pos //= self._sz
        return ''.join(word[::-1])

def commutator(word1: str, word2: str):
    expression = []
    for i in range(len(word1)):
        for j in range(len(word2)):
            # I took the following line from Hartnoll/Xi Yin's code
            word = word1[:i] + word2[j+1:] + word2[:j] + word1[i+1:]
            if word1[i] == 'x' and word2[j] == 'p':
                expression.append('+' + word)
            elif word1[i] == 'p' and word2[j] == 'x':
                expression.append('-' + word)
    return expression

In [ ]:
# Normalization is accounted for in add_linear_constraints

def schwinger_dyson(hamil: list, word1: str):
    # Calculate <[H,O]>. Returns a list of (word, coeff).
    expression = []
    for coeff, w in hamil:
        terms = commutator(w, word1)
        for term in terms:
            if term[0] == '+':
                expression.append((term[1:], coeff))
            else:
                expression.append((term[1:], -coeff))
    return expression

def reality(basis, word: str):
    # Return <trO> = <trO+>* (+ is dagger).
    i = basis.rank(word[::-1])
    j = basis.rank(word)
    return [[(2 * i, 1), (2 * j, -1)], [(2 * i + 1, 1), (2 * j + 1, 1)]]

def symmetry_constraint(G: list, word: str):
    # Calculate <trGO> = 0. Returns a list of (word, coeff).
    return [(w + word, coeff) for coeff, w in G]

def moment_matrix(basis, B, L):
    # Return moment matrix of operators.
    # https://physics.stackexchange.com/questions/130614/complex-semi-definite-programming
    # https://dl.acm.org/doi/10.1145/380752.380838
    n = 2 ** (L//2 + 1) - 1
    Mr = cp.bmat([[B[2 * basis.rank(basis.unrank(i)[::-1] + basis.unrank(j))] for j in range(n)] for i in range(n)])
    Mc = cp.bmat([[B[2 * basis.rank(basis.unrank(i)[::-1] + basis.unrank(j)) + 1] for j in range(n)] for i in range(n)])
    return cp.bmat([[Mr, -Mc], [Mc, Mr]])

def trace_cyclicity(basis, L, word: str):
    # Calculate <trAB> - <trBA>. Returns C in xT@C@x + (this too) <trBA> - <trAB> = 0.
    n = 2 ** (L + 1) - 1
    word1 = word
    word2 = word[1:] + word[0]
    row_indr, col_indr, datar = [], [], []
    row_indc, col_indc, datac = [], [], []
    for i in range(1, len(word)):
        w1ind = basis.rank(word[1:i])
        w2ind = basis.rank(word[i+1:])
        k = 0
        if word[0] == 'x' and word[i] == 'p':
            k = 0.5
        elif word[0] == 'p' and word[i] == 'x':
            k = -0.5
        row_indr.extend([2 * w1ind, 2 * w2ind + 1, 2 * w1ind + 1, 2 * w2ind])
        col_indr.extend([2 * w2ind + 1, 2 * w1ind, 2 * w2ind, 2 * w1ind + 1])
        datar.extend([-k, -k, -k, -k])
        row_indc.extend([2 * w1ind, 2 * w2ind, 2 * w1ind + 1, 2 * w2ind + 1])
        col_indc.extend([2 * w2ind, 2 * w1ind, 2 * w2ind + 1, 2 * w1ind + 1])
        datac.extend([k, k, -k, -k])
    Cr = csr_array((datar, (row_indr, col_indr)), (2 * n, 2 * n))
    Dr = np.zeros((2 * n, 1))
    Dr[2 * basis.rank(word2), 0] = 1
    Dr[2 * basis.rank(word1), 0] = -1
    Cc = csr_array((datac, (row_indc, col_indc)), (2 * n, 2 * n))
    Dc = np.zeros((2 * n, 1))
    Dc[2 * basis.rank(word2) + 1, 0] = 1
    Dc[2 * basis.rank(word1) + 1, 0] = -1
    return [QuadConstraint(Cr, Dr), QuadConstraint(Cc, Dc)]

In [ ]:
def add_linear_constraints(basis, L, hamil, G):
    row_ind, col_ind, data = [0], [0], [1] # This handles the normalization
    numc = 1
    n = 2 ** (L + 1) - 1
    def add_terms(terms):
        nonlocal numc
        new_row = 0
        cnt = 0
        for pos, coeff in terms:
            if pos >= 2 * n:
                return
        for pos, coeff in terms:
            cnt += 1
            new_row = 1
            col_ind.append(pos)
            data.append(coeff)
            row_ind.append(numc)
        numc += new_row
    def preprocess(terms):
        # we have a list terms of form (term, coeff) where term is a string
        # what we have to do separate out the real and complex parts
        # and return a list (i, coeff) where i is the position in the vector B
        realterm = []
        complexterm = []
        for term, coeff in terms:
            i = basis.rank(term)
            cr = np.real(coeff)
            cc = np.imag(coeff)
            realterm.extend([(2 * i, cr), (2 * i + 1, -cc)])
            complexterm.extend([(2 * i, cc), (2 * i + 1, cr)])
        return realterm, complexterm
    # for i in range(n):
    #     word = basis.unrank(i)
    #     realterm, complexterm = preprocess(schwinger_dyson(hamil, word))
    #     add_terms(realterm)
    #     add_terms(complexterm)
    #     realterm, complexterm = preprocess(symmetry_constraint(G, word))
    #     add_terms(realterm)
    #     add_terms(complexterm)
    #     realterm, complexterm = reality(basis, word)
    #     add_terms(realterm)
    #     add_terms(complexterm)
    # Px = b
    P = csr_array((data, (row_ind, col_ind)), (numc, 2 * n))
    b = np.zeros((numc, 1))
    b[0, 0] = 1
    return LinConstraints(P, b)

def add_quad_constraints(basis, L):
    quad_constraints = []
    n = 2 ** (L + 1) - 1
    for i in range(n):
        word = basis.unrank(i)
        if len(word) < 2:
            continue
        quad_constraints.extend(trace_cyclicity(basis, L, word))
    return quad_constraints